Description of constraints and variables..

$c_{i,j,k,l}$ - действие взятия кубика $j$-го цвета $i$-ым манипулятором из $k$-ой кучки $l$-ым номером. ($l$-номером - в смысле, кубик, который этот манипулятор взял первым - имеет первый номер и тд).

Тогда:

$c_{ijkl} \in \{0,1\}$

$c_{ijkl} \geq 0, c_{ijkl} \leq 1$

$\forall i,j,k, ~ l_2 \geq l_1 \Rightarrow c_{ijk,l_2} \geq c_{ijk,l_1}$ - не можем взять второй кубик в манипулятро, пока не взяли первый и тд.

$\forall j,k \sum_{i,l} c_{ijkl} \leq 1$ - берем каждый кубик опр.цвета $j$ из опр. кучки $k$ не более 1го раза

$\forall i,l \sum_{j,k} c_{ijkl} \leq 1$ - в каждом манипуляторе $i$ на опр. высоте $l$ не более 1го кубика

Введем $b_{ijl} = \sum_k c_{ijkl}$ - наличие кубика цвета $j$ в манипуляторе $i$ на высоте $l$ (уже не важно из какой кучки). Для этой переменной можно записать условия на бонус очков в каждом манипуляторе:

Если $a$, $b$, $c$ - бонусная последовательность цветов, то ее можно учесть след. образом для каждого манипулятора.
Пусть $d_i$ - действие выкладывания $i$ манипулятора. Тогда начисление бонусных баллов можно представить в виде переменной $\xi_i$:

$\xi_i \geq 0, \xi_i \leq 1$

У нас есть 6 возможных мест для последовательности из трех кубиков правильного цвета:
наличие на $j$ из 6 мест бонусной посл. в $i$ манипуляторе обозначим за $\pi_{ij}$. Тогда

$\pi_{ij} \geq 0$

$\pi_{i,0} \leq b_{i,0,a}$

$\pi_{i,0} \leq b_{i,0,a}$

$\pi_{i,0} \leq b_{i,1,b}$

$\pi_{i,0} \leq b_{i,2,c}$

и так далее для $\pi_{i,1..}$


Двух бонусных последовательностей в одном манипуляторе быть не может, поэтому
$\xi_i \leq \sum_j \pi_{i,j}$ and $\forall\; i\; \xi_i \in \{0, 1\}$

Соответственно, если есть бонусная посл., то $\xi_i \in \{0,1\}$, а если ее нет, то $\xi_i = 0$. Поэтому, если добавить слагаемое $+A \xi_i$, то $\max...$ будет выбирать его $1$ при возможности.


А для учитывания уже конкретно наличия бонуса в 30 очков остается проверить, выложили ли мы эту башенку:

$\zeta_i \leq \xi_i, \zeta_i \leq d_i, \zeta_i \geq 0$

**obj $ .. + 30\sum_i \zeta_i$ **

Чтобы учесть взятие кубиков с разных сторон, на самом деле, у нас будут переменные $p_{ijklm}$. Здесь $m$ отвечает за то, с какой из 4х сторон мы подъехали к кубику $i$ манипулятором. Не всегда все из этих подъездов возможны. Пусть кубик цвета $j_2$  блокируют подъезд к кубику $j_1$ со стороны $m = 0$. Тогда, 
$index(p_{i,j_1,klm}) \geq \sum_{i,m,l} index(p_{i,j_2,klm})$
Для других цветов пар $j_1$,$j_2$ и подъездов $m$ необходимо написать такие же условия, если блокирует.


Основная переменная - матрица $x_{ij}$.  $x_{ij} = 1$, если j-е действие ($a_j$) является i-ым в порядке выполнения.

$index(a_j) = (1,2,3...)^\top x_{ij}$ 
-дает порядковый номер действия $a_j$, если оно включено в выбранную последовательность. Иначе 0.

Время. Пусть мы тратим на действия $\omega_i$ времени, а на переезды между действиями $\Omega_{ij}$.


$T_1 = \sum_{ij} \sum_N x_{N,i} x_{N-1,j} \Omega_{ij} $,

$T_2 = \omega_j \sum_i x_{ij}$


$T = T_1 + T_2$

objective - формируется за собранные кубики. За каждый кубик дается равно его высоте количество очков, начинается с 1. (+бонус). Запишем в $t_i$ очки за каждую башню:

$t_i = \sum_{j,i} b_{i,j,l}$ ... Но пока $t_i = 5$. И набираемые очки за это фиксированы, максимизировать их не нужно.

$\max_i \sum_i \zeta_i$

бонус +

брать кубики с разных сторон +

выкладываться + 

выкладываться в разных точках ?

разная высота башен -


In [14]:
import numpy as np
import scipy as sp
import scipy.optimize
import cvxpy as cvx
from numpy.random import randn
import matplotlib.pyplot as plt
%matplotlib inline

In [15]:
import gurobipy

In [22]:
pick_action_dims = np.array([3,5,6,5,4])
N = np.prod(pick_action_dims)
print(N)
# 3 manipulators
# 5 colors (and cubes on each pickpoint)
# 6 pickpoints
# 5 places in each manipulator
# 4 directions from which each cube can be picked up

1800


In [19]:
# having for simplicity constraint to load full manipulators, lets also assume we unload once
# so we have N different actions of picking and 1 to unload towers.
# also fake action to with start point

In [20]:
T = 100 #match duration

In [35]:
# picking actions p_i,j,k,l,m where 
# i for manipulator
# j for color
# k for pickpoint
# l for order of picking to each manipulator (first picked - l = 0, second -  l = 1...)
# m for direction..
pick_actions_shifted = np.array([])
print(pick_actions_shifted)
def pick_action_index(i,j,k,l,m): 
    return np.sum(pick_actions_shifted * np.array([i,j,k,l,m]))

[5 6 5 4 1]


In [37]:
pick_action_index(0,0,1,1,1)

10